<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

In [3]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss, ERMGDROLoss, UpweightLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier
from math import ceil
import transformers

Now lets try getting Bert working

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [5]:
df = get_CivilComments_df()

In [6]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [7]:
tr,cv,test = get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [11]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1 / 5
Average training loss: 0.2264478290124004
For Cross Val:
Loss: 0.22557975351810455 Accuracy: 0.9049358123063302 
Accuracy over subgroups: [0.60699301 0.5849546  0.55865922 0.47916667 0.54296875 0.2
 0.67354597 0.62676056 0.68678029 0.92444444 0.93417969 0.83894449
 0.9668747  0.88861076 0.92592593 0.77479893 0.79950372 0.95365328] 
Worst Group Accuracy: 0.2
Epoch 2 / 5
Average training loss: 0.21623695677513527
For Cross Val:
Loss: 0.23032869398593903 Accuracy: 0.8903718459495352 
Accuracy over subgroups: [0.7020979  0.65888457 0.70949721 0.57291667 0.66992188 0.6
 0.79924953 0.77699531 0.72409028 0.88987654 0.90390625 0.70973612
 0.94959193 0.80913642 0.92592593 0.63538874 0.64813896 0.94201763] 
Worst Group Accuracy: 0.5729166666666666
Epoch 3 / 5
Average training loss: 0.2140838625643944
For Cross Val:
Loss: 0.20929887890815735 Accuracy: 0.9173085436033643 
Accuracy over subgroups: [0.4027972  0.40726329 0.34636872 0.26041667 0.33398438 0.
 0.37898687 0.39319249 0.503915

In [12]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

[autoreload of utils.process_data_utils failed: Traceback (most recent call last):
  File "C:\Users\tzeng1\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\tzeng1\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "C:\Program Files\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "C:\Users\tzeng1\repos\ERM_GDRO\utils\process_data_utils.py", line 5, in <module>
    from wilds import get_dataset
ModuleNotFoundError: No module named 'wilds'
]


Accuracy: 0.9160200923891106 
Accuracy over subgroups: [0.52065365 0.50881057 0.53700658 0.4047619  0.43577136 0.21428571
 0.56668835 0.53294746 0.57015825 0.94748594 0.95711968 0.87133956
 0.97686142 0.92810458 0.98019802 0.85007496 0.88956841 0.97650703] 
Worst Group Accuracy: 0.21428571428571427


In [15]:
model.load_state_dict(torch.load('./epoch_2_CC_ERM.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8887742745660852 
Accuracy over subgroups: [0.6895143  0.66035242 0.77878289 0.59047619 0.67301782 0.42857143
 0.80351334 0.79608192 0.71650339 0.88256699 0.90330771 0.68224299
 0.93438559 0.79402428 0.93069307 0.63688156 0.65717281 0.94212067] 
Worst Group Accuracy: 0.42857142857142855


In [13]:
model.load_state_dict(torch.load('./epoch_3_CC_ERM.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.917021721905787 
Accuracy over subgroups: [0.43758511 0.42687225 0.40296053 0.32777778 0.34357714 0.14285714
 0.41314249 0.42742654 0.50293896 0.96543169 0.97355244 0.9211838
 0.98545575 0.95966387 0.99009901 0.92413793 0.93377599 0.98365992] 
Worst Group Accuracy: 0.14285714285714285


In [18]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 18, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_GDRO')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1 / 5
Average training loss: 0.26088099510043666
For Cross Val:
Loss: 0.2877463400363922 Accuracy: 0.8711376715360779 
Accuracy over subgroups: [0.65594406 0.67574578 0.4301676  0.56510417 0.50390625 1.
 0.4727955  0.55868545 0.80239521 0.87259259 0.86308594 0.84804368
 0.92870859 0.84230288 0.85185185 0.83467382 0.82084367 0.90695397] 
Worst Group Accuracy: 0.4301675977653631
Epoch 2 / 5
Average training loss: 0.23896933662595182
For Cross Val:
Loss: 0.26259908080101013 Accuracy: 0.8866533864541832 
Accuracy over subgroups: [0.60559441 0.61089494 0.39106145 0.515625   0.46289062 0.6
 0.44652908 0.5258216  0.78765546 0.90395062 0.90742188 0.90172884
 0.95055209 0.89486859 0.92592593 0.87578195 0.85508685 0.9212145 ] 
Worst Group Accuracy: 0.39106145251396646
Epoch 3 / 5
Average training loss: 0.23078708476787133
For Cross Val:
Loss: 0.2884288430213928 Accuracy: 0.8688800354138999 
Accuracy over subgroups: [0.66713287 0.67444877 0.46648045 0.56770833 0.546875   0.6
 0.51594747 0.6

In [19]:
# model.load_state_dict(torch.load('./epoch_2_CC_GDRO.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8308068350002242 
Accuracy over subgroups: [0.73990014 0.74977974 0.640625   0.69047619 0.68162262 0.64285714
 0.63370202 0.73552983 0.87882442 0.80830301 0.82791452 0.77819315
 0.87794397 0.7318394  0.8019802  0.75892054 0.70697187 0.85128555] 
Worst Group Accuracy: 0.6337020169160703


In [20]:
model.load_state_dict(torch.load('./epoch_4_CC_GDRO.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.843424376971491 
Accuracy over subgroups: [0.72991375 0.74185022 0.61184211 0.68650794 0.62999385 0.5
 0.60572544 0.68744435 0.86284853 0.8213695  0.82988927 0.79034268
 0.88306752 0.77254902 0.79207921 0.77421289 0.74139437 0.86967307] 
Worst Group Accuracy: 0.5


In [9]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM_2')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1 / 5
Average training loss: 0.22609796434781565
For Cross Val:
Loss: 0.2034807950258255 Accuracy: 0.9156485170429394 
Accuracy over subgroups: [0.50909091 0.50194553 0.53631285 0.38802083 0.47851562 0.2
 0.59287054 0.5258216  0.57853524 0.95037037 0.95527344 0.85168335
 0.98007681 0.91739675 0.96296296 0.81680071 0.86054591 0.97629775] 
Worst Group Accuracy: 0.2
Epoch 2 / 5
Average training loss: 0.21557862724064275
For Cross Val:
Loss: 0.20531503856182098 Accuracy: 0.9164674634794157 
Accuracy over subgroups: [0.40839161 0.39429313 0.2849162  0.23697917 0.34179688 0.
 0.54409006 0.43779343 0.51727315 0.96888889 0.97285156 0.94358508
 0.99111858 0.9649562  1.         0.83288651 0.89627792 0.98272282] 
Worst Group Accuracy: 0.0
Epoch 3 / 5
Average training loss: 0.2148581735181079
For Cross Val:
Loss: 0.20218902826309204 Accuracy: 0.9158698539176627 
Accuracy over subgroups: [0.4951049  0.47470817 0.49441341 0.33072917 0.38476562 0.
 0.46904315 0.49295775 0.55734684 0.95580247 0.

In [10]:
# model.load_state_dict(torch.load('./epoch_5_CC_ERM_2.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.9172758667085258 
Accuracy over subgroups: [0.50022696 0.48634361 0.515625   0.38412698 0.43208359 0.28571429
 0.55302537 0.52938557 0.55026375 0.9526133  0.96219762 0.88286604
 0.97950583 0.93501401 0.99009901 0.85997001 0.89079154 0.97978817] 
Worst Group Accuracy: 0.2857142857142857


In [11]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 18, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_GDRO_2')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1 / 5
Average training loss: 0.26006137120775347
For Cross Val:
Loss: 0.32172897458076477 Accuracy: 0.864032757857459 
Accuracy over subgroups: [0.65034965 0.66407263 0.41340782 0.54947917 0.49609375 0.8
 0.51219512 0.57746479 0.82726854 0.87135802 0.87460938 0.88353048
 0.91886702 0.86545682 0.85185185 0.8230563  0.81042184 0.89159647] 
Worst Group Accuracy: 0.4134078212290503
Epoch 2 / 5
Average training loss: 0.2390624364536702
For Cross Val:
Loss: 0.2904393672943115 Accuracy: 0.8708720672864099 
Accuracy over subgroups: [0.63776224 0.67315175 0.41899441 0.56770833 0.484375   0.6
 0.46341463 0.55164319 0.8235836  0.87679012 0.87792969 0.87534122
 0.92558809 0.87609512 0.85185185 0.85969616 0.83573201 0.90021548] 
Worst Group Accuracy: 0.41899441340782123
Epoch 3 / 5
Average training loss: 0.23295992945910493
For Cross Val:
Loss: 0.29892662167549133 Accuracy: 0.8642983621071271 
Accuracy over subgroups: [0.68951049 0.70946822 0.51117318 0.61197917 0.55078125 0.6
 0.52157598 0.6

In [12]:
model.load_state_dict(torch.load('./epoch_5_CC_GDRO_2.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8594130750026162 
Accuracy over subgroups: [0.69087608 0.6938326  0.55345395 0.64285714 0.59065765 0.5
 0.56278465 0.65538736 0.84024115 0.85345683 0.86099161 0.8271028
 0.90306586 0.81195145 0.82178218 0.81529235 0.78507776 0.88731248] 
Worst Group Accuracy: 0.5


In [8]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = UpweightLoss(model, torch.nn.CrossEntropyLoss(), 18, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_UW')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

Epoch 1 / 5
Average training loss: 0.16045779127432652
For Cross Val:
Loss: 0.24272045493125916 Accuracy: 0.9006197432492253 
Accuracy over subgroups: [0.4993007  0.52658885 0.31843575 0.38802083 0.42773438 0.2
 0.47842402 0.51995305 0.71856287 0.93382716 0.94023437 0.92902639
 0.97839654 0.92803504 0.96296296 0.87042002 0.86451613 0.94225269] 
Worst Group Accuracy: 0.2
Epoch 2 / 5
Average training loss: 0.15031163605229683
For Cross Val:
Loss: 0.2443082481622696 Accuracy: 0.8950641876936698 
Accuracy over subgroups: [0.58041958 0.57457847 0.48324022 0.43489583 0.45117188 0.2
 0.53658537 0.55516432 0.73836942 0.92246914 0.92890625 0.86897179
 0.97119539 0.92490613 0.96296296 0.84986595 0.84516129 0.93210578] 
Worst Group Accuracy: 0.2
Epoch 3 / 5
Average training loss: 0.1497923986340751
For Cross Val:
Loss: 0.23583410680294037 Accuracy: 0.900132802124834 
Accuracy over subgroups: [0.55384615 0.53566796 0.49441341 0.39583333 0.4140625  0.2
 0.49530957 0.53638498 0.71718102 0.92938272 0

In [9]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8951129449402759 
Accuracy over subgroups: [0.60689968 0.62070485 0.59292763 0.51031746 0.49170252 0.28571429
 0.6044242  0.60240427 0.75146948 0.91589481 0.92079836 0.84080997
 0.95587142 0.90196078 0.99009901 0.83358321 0.84658396 0.93230349] 
Worst Group Accuracy: 0.2857142857142857


In [10]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = ERMGDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 18, t=0.5, vector_subclass=True)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM_GDRO')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

Epoch 1 / 5
Average training loss: 0.2833691802580899
For Cross Val:
Loss: 0.3833000361919403 Accuracy: 0.83377600708278 
Accuracy over subgroups: [0.6951049  0.68741894 0.61731844 0.57552083 0.65625    0.8
 0.69418386 0.68309859 0.89958544 0.87209877 0.87851563 0.78616924
 0.93086894 0.80600751 0.85185185 0.72832887 0.74640199 0.83428012] 
Worst Group Accuracy: 0.5755208333333334
Epoch 2 / 5
Average training loss: 0.2622008766604013
For Cross Val:
Loss: 0.3767387270927429 Accuracy: 0.834639220894201 
Accuracy over subgroups: [0.71608392 0.73151751 0.67039106 0.60677083 0.66796875 0.6
 0.69606004 0.67957746 0.89405804 0.85185185 0.85078125 0.74340309
 0.92918867 0.78785982 0.85185185 0.71134942 0.74491315 0.84289912] 
Worst Group Accuracy: 0.6
Epoch 3 / 5
Average training loss: 0.2539220160999372
For Cross Val:
Loss: 0.2749040722846985 Accuracy: 0.8767817618415228 
Accuracy over subgroups: [0.56923077 0.56031128 0.5027933  0.45572917 0.53710938 0.6
 0.57410882 0.57159624 0.83049286 0.9

In [11]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8798941561645064 
Accuracy over subgroups: [0.57966409 0.5969163  0.59210526 0.5015873  0.52427781 0.35714286
 0.5862069  0.58637578 0.82351168 0.91978167 0.92030468 0.8258567
 0.95322701 0.88272642 0.97029703 0.83568216 0.84169142 0.9035082 ] 
Worst Group Accuracy: 0.35714285714285715


#Method that better approximates WILDS

Gradient Clip (Norm): 1
AdamW
linear_schedule_with_warmup (warmup): 0

In [ ]:
model = BertClassifier(device=DEVICE)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())

num_training_steps = ceil(len(tr.dataset) / 16) * epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
#note scheduler is in wrong place in train_eval currently

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, scheduler=scheduler, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM_GDRO', gradient_clip = 1)

In [ ]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)